In [12]:
import pandas as pd
from pysam import VariantFile
import numpy as np

# Reduce number of samples using stratified sampling

In [3]:
samples = pd.read_csv('1000genomes_phase3.tsv', sep='\t')
vcf = VariantFile('head2.vcf')

In [4]:
samples = samples[samples['Sample name'].isin(list(vcf.header.samples))]
samples['Superpopulation code'].value_counts()

AFR    666
EUR    517
EAS    508
SAS    492
AMR    347
Name: Superpopulation code, dtype: int64

In [5]:
samples_subset = samples.groupby('Superpopulation code').apply(lambda x: x.sample(n=20))
samples_subset.reset_index(drop=True, inplace=True)
samples_subset.to_csv('samples_subset.tsv', sep='\t', index=False)
num_samples = len(samples_subset)

In [6]:
vcf.subset_samples(list(samples_subset['Sample name']))

# Compute Hamming Distance between subset of samples

In [7]:
num_mismatches = np.zeros((num_samples, num_samples))
num_snps = 0

In [9]:
for rec in vcf.fetch():
    all_homref = True
    tmp_mismatches = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        if rec.samples[i]['GT'] != (0,0):
            all_homref = False
        for j in range(i, num_samples):
            if (rec.samples[i]['GT'] != rec.samples[j]['GT']) & (rec.samples[i]['GT'] != rec.samples[j]['GT'][::-1]):
                tmp_mismatches[i,j] += 1
    if not all_homref:
        num_mismatches += tmp_mismatches
        num_snps += 1

In [10]:
ham_dist = num_mismatches / num_snps
ham_dist = np.triu(ham_dist) + np.tril(ham_dist.T) # Make matrix balanced
np.savetxt('ham_dist.txt', ham_dist, delimiter='\t', fmt='%1.3f')